In [ ]:
import requests
import json
import pandas as pd

### Anwendungsbeispiel 1: Automatisierte Übersetzung mit DeepL

Read the [docs](https://www.deepl.com/docs-api)

In [ ]:
# Abruf Auth Token aus externer Datei
with open('token.json') as f:
    credentials = json.load(f)
    AUTH_KEY_DEEPL = credentials['AUTH_TOKEN_DEEPL']

# Manuelle Eingabe des Auth Token (nicht empfohlen!)
# AUTH_KEY_DEEPL = '<INSERT TOKEN>'

In [ ]:
ENDPOINT = 'https://api-free.deepl.com/v2/translate'

text = """Data literacy is the ability to read, understand, create, and communicate data as information. Much like literacy as a general concept, data literacy focuses on the competencies involved in working with data.
It is, however, not similar to the ability to read text since it requires certain skills involving reading and understanding data."""

HEADER = {'Authorization': 'DeepL-Auth-Key ' + AUTH_KEY_DEEPL}

PARAMS = {'source_lang': 'EN',
          'target_lang': 'DE',
          'text': text}

response = requests.get(ENDPOINT, headers=HEADER, params=PARAMS)
response, HEADER

In [ ]:
response

In [ ]:
response.request.url

In [ ]:
response.json()

In [ ]:
response.json()['translations'][0]['text']

In [ ]:
def translate(text, source_lang, target_lang):
        
    ENDPOINT = 'https://api-free.deepl.com/v2/translate'
    
    with open('token.json') as f:
        credentials = json.load(f)
        AUTH_KEY_DEEPL = credentials['AUTH_TOKEN_DEEPL']

    # AUTH_KEY_DEEPL = '<INSERT TOKEN>'
    
    HEADER = {'Authorization': 'DeepL-Auth-Key ' + AUTH_KEY_DEEPL}

    PARAMS = {'source_lang': 'EN',
              'target_lang': 'DE',
              'text': text}
    
    response = requests.get(ENDPOINT, headers=HEADER, params=PARAMS).json()['translations'][0]['text']
    return response

In [ ]:
sentences = ["Data literacy is the ability to read, understand, create, and communicate data as information.",
             "Much like literacy as a general concept, data literacy focuses on the competencies involved in working with data.",
             "It is, however, not similar to the ability to read text since it requires certain skills involving reading and understanding data."]

for sent in sentences:
    print(translate(sent, 'EN', 'DE'))

*Hinweis: Es gibt sogar ein separates Python package (`deepl`), was die Benutzung der API noch einfacher macht!*

### Anwendungsbeispiel 2: Transkription mit AssemblyAI

Read the [docs](https://docs.assemblyai.com/)

Schritt 1: Upload der Audio-Datei

In [ ]:
UPLOAD_ENDPOINT = 'https://api.assemblyai.com/v2/upload'

with open('token.json') as f:
    credentials = json.load(f)
    AUTH_TOKEN_ASSEMBLYAI = credentials['AUTH_TOKEN_ASSEMBLYAI']

# AUTH_TOKEN_ASSEMBLYAI = '<INSERT TOKEN>'

HEADER = {'authorization': AUTH_TOKEN_ASSEMBLYAI,
          'content-type': 'application/json'}

with open('recording.m4a', 'rb') as f:
    audio_file = f.read()
    
response = requests.post(UPLOAD_ENDPOINT, headers=HEADER, data=audio_file)

In [ ]:
response

In [ ]:
response.json()

Schritt 2: Starten der Transkription

Inzwischen werden auch Transkriptionen in [Spanisch, Französisch, Deutsch und Italienisch](https://docs.assemblyai.com/#supported-languages) unterstützt.

In [ ]:
TRANSCRIPT_ENDPOINT = 'https://api.assemblyai.com/v2/transcript'

UPLOAD_URL = response.json()['upload_url']

JSON = {'audio_url': UPLOAD_URL,
        'language_code': 'en',
        'speaker_labels': 'true'}

reponse = requests.post(TRANSCRIPT_ENDPOINT, headers=HEADER, json=JSON)

In [ ]:
response

In [ ]:
reponse.json()

Schritt 3: Download des Transkriptes

*Hinweis: Laut [Anbieter](https://docs.assemblyai.com/#processing-times) veranschlagt der Prozess etwa 30% der Länge der Audio-Datei.*

In [ ]:
TRANSCRIPT_ID = reponse.json()['id']

response = requests.get(TRANSCRIPT_ENDPOINT + '/' + TRANSCRIPT_ID, headers=HEADER)
response.json()

In [ ]:
response.json()['text']

### Anwendungsbeispiel 3: Musik-Empfehlungen mit Spotify

Read the [docs](https://developer.spotify.com/documentation/web-api/)

Schritt 1: Erstellen eines Authentication Token

In [ ]:
AUTH_ENDPOINT = 'https://accounts.spotify.com/api/token'

with open('token.json') as f:
    credentials = json.load(f)
    CLIENT_ID_SPOTIFY = credentials['CLIENT_ID_SPOTIFY']
    CLIENT_SECRET_SPOTIFY = credentials['CLIENT_SECRET_SPOTIFY'] 

# CLIENT_ID_SPOTIFY = '<INSERT CLIENT_ID>'
# CLIENT_SECRET_SPOTIFY = '<INSERT CLIENT_SECRET>'

DATA = {'grant_type': 'client_credentials',
        'client_id': CLIENT_ID_SPOTIFY,
        'client_secret': CLIENT_SECRET_SPOTIFY}

response = requests.post(AUTH_ENDPOINT, data=DATA)

In [ ]:
response

In [ ]:
response.json()

Schritt 2: Abruf von Song Eigenschaften

*Hinweis: Die Spotify API bietet eine Fülle an verschiedenen Abfragemöglichkeiten (d.h. [Endpoints](https://developer.spotify.com/documentation/web-api/reference/#/)). Zudem gibt es ein separates Python package (`spotipy`), was die Benutzung der API noch einfacher macht!*

In [ ]:
FEATURES_ENDPOINT = 'https://api.spotify.com/v1/audio-features/'

AUTH_TOKEN_SPOTIFY = response.json()['access_token']

TRACK_ID = '1bBglCIHeIwR78kJXJzkZo'

HEADER = {'Content-Type': 'application/json',
           'Authorization': 'Bearer ' + AUTH_TOKEN_SPOTIFY}

response = requests.get(FEATURES_ENDPOINT + TRACK_ID, headers=HEADER)

In [ ]:
response

In [ ]:
response.json()

Schritt 3: Abruf von Song Empfehlungen

In [ ]:
RECOMMEND_ENDPOINT = 'https://api.spotify.com/v1/recommendations'

PARAMS = {'seed_tracks': '1bBglCIHeIwR78kJXJzkZo,30dDeLflXjGSv7qjrjmYCG,6gOR8xTSQWEMfEJa7r52k7',
          'limit': 5}

response = requests.get(RECOMMEND_ENDPOINT, headers=HEADER, params=PARAMS)

In [ ]:
response

In [ ]:
response.json()

### Anwendungsbeispiel 4: Amtliche Statistiken mit Datenguide

Read the [docs](https://datengui.de/docs)

*Hinweis: Da Datenguide open data bereitstellt brauchen wir keine `auth_token`! Der `data` Code `52411:ISV001` kennzeichnet den Datensatz "Statistik über beantragte Insolvenzverfahren - Insolvenzverfahren", der `region` Code `05` steht für "Nordrhein-Westfalen", und es werden alle verfügbaren Daten ab `2008` abgerufen.*

In [ ]:
ENDPOINT_DATENGUIDE = 'https://tabular.genesapi.org'

PARAMS = {'data': '52411:ISV001',
          'region': '05',
          'time': '2008:',
          'format':'json'}

response = requests.get(ENDPOINT_DATENGUIDE, params=PARAMS)

In [ ]:
response

In [ ]:
response.json()

In [ ]:
pd.DataFrame(response.json()['data'])